# Candlesticks Combined patterns analysis

In [91]:
import pandas as pd
df = pd.read_csv(r"C:\Users\Sid\Desktop\vit\DataScience Projects\Dataset\AAPL.csv")
df.tail()


,Date,Open,High,Low,Close,Adj Close,Volume
2770,2023-01-05,127.129997,127.769997,124.760002,125.019997,125.019997,80962700
2771,2023-01-06,126.010002,130.289993,124.889999,129.619995,129.619995,87686600
2772,2023-01-09,130.470001,133.410004,129.889999,130.149994,130.149994,70790800
2773,2023-01-10,130.259995,131.259995,128.119995,130.729996,130.729996,63896200
2774,2023-01-11,131.250000,133.509995,130.460007,133.490005,133.490005,69359700


In [92]:
df.shape


(2775, 7)

In [81]:
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2012-01-03,14.621429,14.732143,14.607143,14.686786,12.519278,302220800
1,2012-01-04,14.642857,14.810000,14.617143,14.765714,12.586558,260022000
2,2012-01-05,14.819643,14.948214,14.738214,14.929643,12.726296,271269600
3,2012-01-06,14.991786,15.098214,14.972143,15.085714,12.859330,318292800
4,2012-01-09,15.196429,15.276786,15.048214,15.061786,12.838933,394024400


In [93]:
#Check if NA values are in data
df.isna().sum()
df= df[df['Volume'] != 0]
df.reset_index(drop=True, inplace=True)

import pandas_ta as ta
#df.ta.indicators()
#help(ta.atr)
#df['ATR'] = df.ta.atr(length=10)
df['RSI'] = df.ta.rsi()
df.tail()

,Date,Open,High,Low,Close,Adj Close,Volume,RSI
2770,2023-01-05,127.129997,127.769997,124.760002,125.019997,125.019997,80962700,33.467992
2771,2023-01-06,126.010002,130.289993,124.889999,129.619995,129.619995,87686600,42.118917
2772,2023-01-09,130.470001,133.410004,129.889999,130.149994,130.149994,70790800,43.037926
2773,2023-01-10,130.259995,131.259995,128.119995,130.729996,130.729996,63896200,44.084227
2774,2023-01-11,131.250000,133.509995,130.460007,133.490005,133.490005,69359700,48.894849


In [94]:
length = len(df)
high = list(df['High'])
low = list(df['Low'])
close = list(df['Close'])
open = list(df['Open'])
bodydiff = [0] * length

highdiff = [0] * length
lowdiff = [0] * length
ratio1 = [0] * length
ratio2 = [0] * length

def isEngulfing(l):
    row=l
    bodydiff[row] = abs(open[row]-close[row])
    if bodydiff[row]<0.000001:
        bodydiff[row]=0.000001      

    bodydiffmin = 0.002
    if (bodydiff[row]>bodydiffmin and bodydiff[row-1]>bodydiffmin and
        open[row-1]<close[row-1] and
        open[row]>close[row] and 
        (open[row]-close[row-1])>=-0e-5 and close[row]<open[row-1]): #+0e-5 -5e-5
        return 1

    elif(bodydiff[row]>bodydiffmin and bodydiff[row-1]>bodydiffmin and
        open[row-1]>close[row-1] and
        open[row]<close[row] and 
        (open[row]-close[row-1])<=+0e-5 and close[row]>open[row-1]):#-0e-5 +5e-5
        return 2
    else:
        return 0
    
    
def isEngulfingStrong(l):
    row=l
    bodydiff[row] = abs(open[row]-close[row])
    if bodydiff[row]<0.000001:
        bodydiff[row]=0.000001      

    bodydiffmin = 0.002
    if (bodydiff[row]>bodydiffmin and bodydiff[row-1]>bodydiffmin and
        open[row-1]<close[row-1] and
        open[row]>close[row] and 
        (open[row]-close[row-1])>=-0e-5 and close[row]<low[row-1]): #+0e-5 -5e-5
        return 1

    elif(bodydiff[row]>bodydiffmin and bodydiff[row-1]>bodydiffmin and
        open[row-1]>close[row-1] and
        open[row]<close[row] and 
        (open[row]-close[row-1])<=+0e-5 and close[row]>high[row-1]):#-0e-5 +5e-5
        return 2
    else:
        return 0
    
def isStar(l):
    bodydiffmin = 0.0020
    row=l
    highdiff[row] = high[row]-max(open[row],close[row])
    lowdiff[row] = min(open[row],close[row])-low[row]
    bodydiff[row] = abs(open[row]-close[row])
    if bodydiff[row]<0.000001:
        bodydiff[row]=0.000001
    ratio1[row] = highdiff[row]/bodydiff[row]
    ratio2[row] = lowdiff[row]/bodydiff[row]

    if (ratio1[row]>1 and lowdiff[row]<0.2*highdiff[row] and bodydiff[row]>bodydiffmin):# and open[row]>close[row]):
        return 1
    elif (ratio2[row]>1 and highdiff[row]<0.2*lowdiff[row] and bodydiff[row]>bodydiffmin):# and open[row]<close[row]):
        return 2
    else:
        return 0

def direction(l):
    if (open[l]>close[l]):
        return 1
    elif (open[l]<close[l]):
        return 2
    else:
        return 0
    

# Identifying the candle signal

In [105]:
def Revsignal1():
    signal = [0] * length
    for row in range(1, length):
        if ((isEngulfing(row)==1 or isStar(row)==1) and df.RSI[row]<30):
            signal[row] = 1
        elif((isEngulfing(row)==2 or isStar(row)==2) and df.RSI[row]>70):
            signal[row] = 2
        else:
            signal[row] = 0
    return signal

df['signal'] = Revsignal1()
df.tail()

,Date,Open,High,Low,Close,Adj Close,Volume,RSI,signal,Trend
2770,2023-01-05,127.129997,127.769997,124.760002,125.019997,125.019997,80962700,33.467992,0,NaN
2771,2023-01-06,126.010002,130.289993,124.889999,129.619995,129.619995,87686600,42.118917,0,NaN
2772,2023-01-09,130.470001,133.410004,129.889999,130.149994,130.149994,70790800,43.037926,0,NaN
2773,2023-01-10,130.259995,131.259995,128.119995,130.729996,130.729996,63896200,44.084227,0,NaN
2774,2023-01-11,131.250000,133.509995,130.460007,133.490005,133.490005,69359700,48.894849,0,NaN


In [107]:
#df.reset_index(drop=True, inplace=True)
df[df['signal']==2].count()

Date         53
Open         53
High         53
Low          53
Close        53
Adj Close    53
Volume       53
RSI          53
signal       53
Trend        53
dtype: int64

In [108]:
#Target
def mytarget(df1, barsfront):
    length = len(df1)
    high = list(df1['High'])
    low = list(df1['Low'])
    close = list(df1['Close'])
    open = list(df1['Open'])
    trendcat = [None] * length
    
    piplim = 300e-5
    for line in range (0, length-1-barsfront):
        for i in range(1,barsfront+1):
            if( ((high[line+i]-close[line])>piplim) and ((close[line]-low[line+i])>piplim) ):
                trendcat[line] = 3 # no trend
                break
            elif(close[line]-low[line+i])>piplim:
                trendcat[line] = 1 #-1 downtrend
                break
            elif ((high[line+i]-close[line])>piplim):
                trendcat[line] = 2 # uptrend
                break
            else:
                trendcat[line] = 0 # no clear trend  
    return trendcat

df['Trend'] = mytarget(df,10)
df.head(30)

,Date,Open,High,Low,Close,Adj Close,Volume,RSI,signal,Trend
0,2012-01-03,14.621429,14.732143,14.607143,14.686786,12.519278,302220800,NaN,0,3.0
1,2012-01-04,14.642857,14.810000,14.617143,14.765714,12.586558,260022000,NaN,0,3.0
2,2012-01-05,14.819643,14.948214,14.738214,14.929643,12.726296,271269600,NaN,0,2.0
3,2012-01-06,14.991786,15.098214,14.972143,15.085714,12.859330,318292800,NaN,0,3.0
4,2012-01-09,15.196429,15.276786,15.048214,15.061786,12.838933,394024400,NaN,0,3.0
5,2012-01-10,15.211071,15.214286,15.053571,15.115714,12.884902,258196400,NaN,0,1.0
6,2012-01-11,15.095714,15.101786,14.975357,15.091071,12.863899,215084800,NaN,0,3.0
7,2012-01-12,15.081429,15.103571,14.955357,15.049643,12.828583,212587200,NaN,0,1.0
8,2012-01-13,14.989286,15.016071,14.952143,14.993214,12.780482,226021600,NaN,0,2.0
9,2012-01-17,15.150000,15.213929,15.105714,15.167857,12.929351,242897200,NaN,0,2.0


In [113]:
import numpy as np
conditions = [( (df['Trend'] == 2) | (df['Trend'] == 3) ) & (df['signal'] == 1), ((df['Trend'] == 2) | (df['Trend'] == 3)) & (df['signal'] == 2)]
values = [1, 2]
df['result'] = np.select(conditions, values)

trendId=1
print(df[df['result']==trendId].result.count()/df[df['signal']==trendId].signal.count())
df[ (df['Trend']!=trendId) & (df['Trend']!=3) & (df['signal']==trendId) ] # false positives

0.875


,Date,Open,High,Low,Close,Adj Close,Volume,RSI,signal,Trend,result
1097,2016-05-13,22.500000,22.917500,22.500000,22.629999,20.956589,177571200,23.427295,1,2.0,1
2302,2021-02-26,122.589996,124.849998,121.199997,121.260002,120.000671,164560400,29.106771,1,2.0,1


In [111]:
dfpl = df[2700:2775]
import plotly.graph_objects as go
from datetime import datetime

fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['Open'],
                high=dfpl['High'],
                low=dfpl['Low'],
                close=dfpl['Close'])])

fig.show()